### Intro :
1. We need to reduce dimensionality of cell and gene features !
2. Most public Kernels pick a Number and do Pca , while this method works well there is another much more reliable and most probably .
3. The Method can be found [Here](https://www.appliedaicourse.com/lecture/11/applied-machine-learning-online-course/2896/pca-for-dimensionality-reduction-not-visualization/2/module-2-data-science-exploratory-data-analysis-and-data-visualization)
4. Do let me know in the comments if it offers some improvements! Also please do upvote kernel if you find it useful!

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings("ignore")



import os
import gc
import pickle
import datetime
import numpy as np
import pandas as pd
import xgboost as xgb
from tqdm.notebook import tqdm
from time import time

In [ ]:
from sklearn.preprocessing import StandardScaler
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')

ss0 = pd.read_csv('../input/lish-moa/sample_submission.csv') 
ss = pd.read_csv('../input/lish-moa/sample_submission.csv') 

cols = [c for c in ss.columns.values if c != 'sig_id']
GENES = [col for col in train_features.columns if col.startswith('g-')] # gene columns
CELLS = [col for col in train_features.columns if col.startswith('c-')] # cell Columns

In [ ]:
def preprocess(df):
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    del df['sig_id']
    return df

def metric(y_true, y_pred):
    metrics = []
    for _target in train_targets.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float), labels=[0,1]))
    return np.mean(metrics)

train = preprocess(train_features)
test = preprocess(test_features)

del train_targets['sig_id']
del train_targets_nonscored['sig_id']

## 1. Optimal Components for Gene Features

In [ ]:
from sklearn.preprocessing import StandardScaler
# Quantile Transform is used to Map Each Feature to a Normal Distribution 
# This transformation tends to spread out the most frequent values and also reduces impact of Outliers
std_scalar = StandardScaler()
# Fit Quantile to both Train and Test
std_scalar.fit(pd.concat([pd.DataFrame(train[GENES]), pd.DataFrame(test[GENES])]))
# Transform Train and Test Cell and Gene Features Individually
train[GENES] = std_scalar.transform(train[GENES])
test[GENES] = std_scalar.transform(test[GENES])

In [ ]:
train[GENES].shape

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
pca = PCA()
pca.n_components = 772
data = pd.concat([pd.DataFrame(train[GENES]), pd.DataFrame(test[GENES])])
pca_data = pca.fit_transform(data)
percentage_var_explained = pca.explained_variance_ / np.sum(pca.explained_variance_);

cum_var_explained = np.cumsum(percentage_var_explained)

# Plot the PCA spectrum
plt.figure(1, figsize=(6, 4))

plt.clf()
plt.plot(cum_var_explained, linewidth=2)
plt.axis('tight')
plt.grid()
plt.xlabel('n_components')
plt.ylabel('Cumulative_explained_variance')
plt.show()

1. We can see just by using 400 Components of Gene Features we are able to Explain 90% Variance
2. By using 500 features we can explain 95 % of Variance

## 2. Optimal Components for Cell Features 

In [ ]:
from sklearn.preprocessing import StandardScaler
# Quantile Transform is used to Map Each Feature to a Normal Distribution 
# This transformation tends to spread out the most frequent values and also reduces impact of Outliers
std_scalar = StandardScaler()
# Fit Quantile to both Train and Test
std_scalar.fit(pd.concat([pd.DataFrame(train[CELLS]), pd.DataFrame(test[CELLS])]))
# Transform Train and Test Cell and Gene Features Individually
train[CELLS] = std_scalar.transform(train[CELLS])
test[CELLS] = std_scalar.transform(test[CELLS])

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
pca = PCA()
pca.n_components = 100
data = pd.concat([pd.DataFrame(train[CELLS]), pd.DataFrame(test[CELLS])])
pca_data = pca.fit_transform(data)
percentage_var_explained = pca.explained_variance_ / np.sum(pca.explained_variance_);

cum_var_explained = np.cumsum(percentage_var_explained)

# Plot the PCA spectrum
plt.figure(1, figsize=(6, 4))

plt.clf()
plt.plot(cum_var_explained, linewidth=2)
plt.axis('tight')
plt.grid()
plt.xlabel('n_components')
plt.ylabel('Cumulative_explained_variance')
plt.show()

1. We can see that by using 80 principal components we can explain nearly more than 98 % of Variance